## Description

Bone models of 16S rRNA data at all taxonomic levels, using the ml_new environment, rarefied data and no normalization during import, n = 1000 estimators, k fold = 6 (for regular, non-nested cross-validation), and hyperparameter tuning.

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing, svm, metrics
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, GroupKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_error, roc_auc_score, precision_score, make_scorer
from sklearn.linear_model import LassoCV, Lasso, LassoLarsIC, ElasticNet, LassoLarsCV
from sklearn.externals import joblib
from scipy.stats import randint as sp_randint
from matplotlib import pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

/Users/heatherdeel/miniconda3/envs/ml_new/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
%matplotlib inline
import biom
import calour as ca
from scipy import stats
import scipy
import pickle
import time
import math
import inspect
pd.set_option('display.max_rows', 10000)

/Users/heatherdeel/miniconda3/envs/ml_new/lib/python3.7/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt


## Import rarefied data collapsed at L2

In [3]:
exp_L2 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/01_qiime2_analysis/feature_tables/collapsed_tables/frag_ins_noChloMito_17098_L2_table.biom', '../02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 11:12:10 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [4]:
exp_L2.sample_metadata['host_subject_id'].value_counts()

STAFS2016.065    8
STAFS2016.011    8
STAFS2016.067    8
STAFS2016.007    7
STAFS2016.064    7
STAFS2016.024    7
Name: host_subject_id, dtype: int64

In [5]:
exp_L2.sample_metadata['ADD_0']

#SampleID
11553.SHSU.STAFS2016.024.R09      708.0555556
11553.STAFS.2016.065.L12          2268.055556
11553.STAFS.2016.067.R12          2414.444444
11553.SHSU.STAFS2016.007.L09      1911.111111
11553.SHSU.STAFS2016.011.L08      1301.666667
11553.STAFS.2016.065.R11          4756.388889
11553.STAFS.2016.065.R09          3456.111111
11553.SHSU.STAFS2016.024.L12      4285.833333
11553.SHSU.STAFS2016.007.L11      2584.444444
11553.SHSU.STAFS2016.011.L10      3777.222222
11553.STAFS.2016.064.R10          3456.111111
11553.SHSU.STAFS2016.024.R11      2584.444444
11553.STAFS.2016.067.L11.june     5201.388889
11553.SHSU.STAFS2016.007.R12      1301.666667
11553.STAFS.2016.065.R10          2804.722222
11553.STAFS.2016.064.R09          592.7777778
11553.STAFS.2016.067.L10          3623.055556
11553.SHSU.STAFS2016.011.R12      708.0555556
11553.STAFS.2016.064.R11          1300.555556
11553.SHSU.STAFS2016.011.L11      4285.833333
11553.SHSU.STAFS2016.007.L10      4821.388889
11553.STAFS.2016.065.R08

In [6]:
print(exp_L2.feature_metadata)

                                                     _feature_id
k__Archaea;p__Crenarchaeota          k__Archaea;p__Crenarchaeota
k__Archaea;p__Euryarchaeota          k__Archaea;p__Euryarchaeota
k__Bacteria;__                                    k__Bacteria;__
k__Bacteria;p__Acidobacteria        k__Bacteria;p__Acidobacteria
k__Bacteria;p__Actinobacteria      k__Bacteria;p__Actinobacteria
k__Bacteria;p__Armatimonadetes    k__Bacteria;p__Armatimonadetes
k__Bacteria;p__BRC1                          k__Bacteria;p__BRC1
k__Bacteria;p__Bacteroidetes        k__Bacteria;p__Bacteroidetes
k__Bacteria;p__Chlorobi                  k__Bacteria;p__Chlorobi
k__Bacteria;p__Chloroflexi            k__Bacteria;p__Chloroflexi
k__Bacteria;p__Cyanobacteria        k__Bacteria;p__Cyanobacteria
k__Bacteria;p__Deferribacteres    k__Bacteria;p__Deferribacteres
k__Bacteria;p__FBP                            k__Bacteria;p__FBP
k__Bacteria;p__Firmicutes              k__Bacteria;p__Firmicutes
k__Bacteria;p__Fusobacter

## Running the L2 model

In [7]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_L2.data
y = exp_L2.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_L2.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [8]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11c24bd50>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [9]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [10]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 807.77


In [11]:
joblib.dump(gs.best_estimator_, 'models/bone_L2_no_metadata.pkl')

['models/bone_L2_no_metadata.pkl']

In [13]:
bone_L2 = joblib.load('models/bone_L2_no_metadata.pkl')
bone_L2

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L2 model

In [14]:
importances = bone_L2.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_L2.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_L2.feature_metadata
np.savetxt("importances/bone_L2_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_L2.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_L2.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(58,1))

np.savetxt("importances/bone_L2_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. k__Bacteria;p__Bacteroidetes	0.12898262251478135
2. k__Bacteria;p__Firmicutes	0.10654712402856573
3. k__Bacteria;p__Proteobacteria	0.10439694161802271
4. k__Bacteria;p__Planctomycetes	0.10202943289184362
5. k__Bacteria;p__Verrucomicrobia	0.08525484161578097
6. k__Bacteria;p__Actinobacteria	0.05857810856832839
7. k__Bacteria;p__Chloroflexi	0.050709857537267734
8. k__Bacteria;p__Gemmatimonadetes	0.05059621932991354
9. k__Bacteria;p__Fusobacteria	0.048826247485686314
10. k__Bacteria;p__[Thermi]	0.04733001573440782
11. k__Bacteria;p__Acidobacteria	0.03431768224393029
12. k__Bacteria;p__FBP	0.026856367078377463
13. k__Bacteria;p__BRC1	0.024707051693709556
14. k__Bacteria;p__Cyanobacteria	0.017336468579629335
15. k__Bacteria;p__GN02	0.016742866760758156
16. k__Bacteria;p__Tenericutes	0.01588992806520978
17. k__Bacteria;p__Armatimonadetes	0.01350504503507785
18. k__Bacteria;p__Deferribacteres	0.013485275346605171
19. k__Bacteria;p__TM7	0.012013872490631803
20. k__Ba

## Import rarefied table collapsed at L3

In [15]:
exp_L3 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/01_qiime2_analysis/feature_tables/collapsed_tables/frag_ins_noChloMito_17098_L3_table.biom', '/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 12:58:12 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [16]:
exp_L3.sample_metadata['host_subject_id'].value_counts()

STAFS2016.065    8
STAFS2016.011    8
STAFS2016.067    8
STAFS2016.007    7
STAFS2016.064    7
STAFS2016.024    7
Name: host_subject_id, dtype: int64

In [17]:
exp_L3.sample_metadata['ADD_0']

#SampleID
11553.SHSU.STAFS2016.024.R09      708.0555556
11553.STAFS.2016.065.L12          2268.055556
11553.STAFS.2016.067.R12          2414.444444
11553.SHSU.STAFS2016.007.L09      1911.111111
11553.SHSU.STAFS2016.011.L08      1301.666667
11553.STAFS.2016.065.R11          4756.388889
11553.STAFS.2016.065.R09          3456.111111
11553.SHSU.STAFS2016.024.L12      4285.833333
11553.SHSU.STAFS2016.007.L11      2584.444444
11553.SHSU.STAFS2016.011.L10      3777.222222
11553.STAFS.2016.064.R10          3456.111111
11553.SHSU.STAFS2016.024.R11      2584.444444
11553.STAFS.2016.067.L11.june     5201.388889
11553.SHSU.STAFS2016.007.R12      1301.666667
11553.STAFS.2016.065.R10          2804.722222
11553.STAFS.2016.064.R09          592.7777778
11553.STAFS.2016.067.L10          3623.055556
11553.SHSU.STAFS2016.011.R12      708.0555556
11553.STAFS.2016.064.R11          1300.555556
11553.SHSU.STAFS2016.011.L11      4285.833333
11553.SHSU.STAFS2016.007.L10      4821.388889
11553.STAFS.2016.065.R08

In [18]:
print(exp_L3.feature_metadata)

                                                                                          _feature_id
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota           k__Archaea;p__Crenarchaeota;c__Thaumarchaeota
k__Archaea;p__Euryarchaeota;c__Methanobacteria         k__Archaea;p__Euryarchaeota;c__Methanobacteria
k__Bacteria;__;__                                                                   k__Bacteria;__;__
k__Bacteria;p__Acidobacteria;c__DA052                           k__Bacteria;p__Acidobacteria;c__DA052
k__Bacteria;p__Acidobacteria;c__Solibacteres             k__Bacteria;p__Acidobacteria;c__Solibacteres
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia       k__Bacteria;p__Actinobacteria;c__Acidimicrobiia
k__Bacteria;p__Actinobacteria;c__Actinobacteria       k__Bacteria;p__Actinobacteria;c__Actinobacteria
k__Bacteria;p__Actinobacteria;c__Coriobacteriia       k__Bacteria;p__Actinobacteri

## Running the L3 model

In [19]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_L3.data
y = exp_L3.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_L3.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [20]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11bee87d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [21]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': True, 'max_depth': 16, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [22]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 834.43


In [23]:
joblib.dump(gs.best_estimator_, 'models/bone_L3_no_metadata.pkl')

['models/bone_L3_no_metadata.pkl']

In [24]:
bone_L3 = joblib.load('models/bone_L3_no_metadata.pkl')
bone_L3

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=16, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L3 model

In [26]:
importances = bone_L3.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_L3.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_L3.feature_metadata
np.savetxt("importances/bone_L3_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_L3.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_L3.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(124,1))

np.savetxt("importances/bone_L3_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria	0.07733366143688698
2. k__Bacteria;p__Bacteroidetes;c__Sphingobacteriia	0.07336246704885896
3. k__Bacteria;p__Bacteroidetes;c__[Saprospirae]	0.05617383007595434
4. k__Bacteria;p__Actinobacteria;c__Thermoleophilia	0.04871364610607001
5. k__Bacteria;p__Proteobacteria;c__Betaproteobacteria	0.04849792815677068
6. k__Bacteria;p__Bacteroidetes;c__Flavobacteriia	0.04703361022708913
7. k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria	0.039783972519649984
8. k__Bacteria;p__Firmicutes;c__Bacilli	0.03878517683151592
9. k__Bacteria;p__Firmicutes;c__Clostridia	0.03874159789314058
10. k__Bacteria;p__Actinobacteria;c__Acidimicrobiia	0.03862518272523338
11. k__Bacteria;p__Planctomycetes;c__Planctomycetia	0.03709047884911729
12. k__Bacteria;p__Verrucomicrobia;c__[Spartobacteria]	0.03235828387417305
13. k__Bacteria;p__Firmicutes;c__Erysipelotrichi	0.030601402311212916
14. k__Bacteria;p__Bacteroidetes;c__Bacteroidia	0.029

## Import rarefied table collapsed at L4

In [27]:
exp_L4 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/01_qiime2_analysis/feature_tables/collapsed_tables/frag_ins_noChloMito_17098_L4_table.biom', '../02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 13:23:49 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [28]:
print(exp_L4.feature_metadata)

                                                                                          _feature_id
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o...  k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;...
k__Archaea;p__Euryarchaeota;c__Methanobacteria;...  k__Archaea;p__Euryarchaeota;c__Methanobacteria...
k__Bacteria;__;__;__                                                             k__Bacteria;__;__;__
k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin6513  k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin...
k__Bacteria;p__Acidobacteria;c__Solibacteres;o_...  k__Bacteria;p__Acidobacteria;c__Solibacteres;o...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;c__Acidimicrobii...
k__Bacteria;p__Actinobacteria;c__Actinobacteria...  k__Bacteria;p__Actinobacteria;c__Actinobacteri...
k__Bacteria;p__Actinobacteria;c__Coriobacteriia...  k__Bacteria;p__Actinobacteria;

## Running the L4 model

In [29]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_L4.data
y = exp_L4.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_L4.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [30]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11cdbd550>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [31]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [32]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 839.88


In [33]:
joblib.dump(gs.best_estimator_, 'models/bone_L4_no_metadata.pkl')

['models/bone_L4_no_metadata.pkl']

In [34]:
bone_L4 = joblib.load('models/bone_L4_no_metadata.pkl')
bone_L4

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L4 model

In [36]:
importances = bone_L4.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_L4.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_L4.feature_metadata
np.savetxt("importances/bone_L4_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_L4.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_L4.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(258,1))

np.savetxt("importances/bone_L4_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. k__Bacteria;p__Bacteroidetes;c__Sphingobacteriia;o__Sphingobacteriales	0.05164475913473576
2. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Caulobacterales	0.04386094769789566
3. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales	0.042179583901854306
4. k__Bacteria;p__Actinobacteria;c__Thermoleophilia;o__Solirubrobacterales	0.04210819413389361
5. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Sphingomonadales	0.033972312379367445
6. k__Bacteria;p__Bacteroidetes;c__[Saprospirae];o__[Saprospirales]	0.032130708707649454
7. k__Bacteria;p__Bacteroidetes;c__Flavobacteriia;o__Flavobacteriales	0.03203002339654623
8. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodospirillales	0.031490410708379026
9. k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholderiales	0.029990158416624783
10. k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Xanthomonadales	0.02708688929066615
11. k__Bacteria;p__Gemmatimonadetes;

## Import rarefied table collapsed at L5

In [37]:
exp_L5 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/01_qiime2_analysis/feature_tables/collapsed_tables/frag_ins_noChloMito_17098_L5_table.biom', '../02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 13:49:48 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [38]:
print(exp_L5.feature_metadata)

                                                                                          _feature_id
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o...  k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;...
k__Archaea;p__Euryarchaeota;c__Methanobacteria;...  k__Archaea;p__Euryarchaeota;c__Methanobacteria...
k__Bacteria;__;__;__;__                                                       k__Bacteria;__;__;__;__
k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin6...  k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin...
k__Bacteria;p__Acidobacteria;c__Solibacteres;o_...  k__Bacteria;p__Acidobacteria;c__Solibacteres;o...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;c__Acidimicrobii...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;

## Running the L5 model

In [39]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_L5.data
y = exp_L5.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_L5.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [40]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11c35a950>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [41]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': True, 'max_depth': 8, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [42]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 825.62


In [43]:
joblib.dump(gs.best_estimator_, 'models/bone_L5_no_metadata.pkl')

['models/bone_L5_no_metadata.pkl']

In [44]:
bone_L5 = joblib.load('models/bone_L5_no_metadata.pkl')
bone_L5

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L5 model

In [46]:
importances = bone_L5.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_L5.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_L5.feature_metadata
np.savetxt("importances/bone_L5_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_L5.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_L5.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(516,1))

np.savetxt("importances/bone_L5_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Phyllobacteriaceae	0.03718643766076944
2. k__Bacteria;p__Bacteroidetes;c__Sphingobacteriia;o__Sphingobacteriales;f__Sphingobacteriaceae	0.025144454188960896
3. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Bradyrhizobiaceae	0.024502459033168902
4. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodospirillales;f__Acetobacteraceae	0.02266622298647285
5. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Caulobacterales;f__Caulobacteraceae	0.022392539797286223
6. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Rhizobiaceae	0.020892859383385826
7. k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Promicromonosporaceae	0.020589386829643154
8. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Brucellaceae	0.019999697221153002
9. k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Burkholder

## Import rarefied table collapsed at L6

In [47]:
exp_L6 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/01_qiime2_analysis/feature_tables/collapsed_tables/frag_ins_noChloMito_17098_L6_table.biom', '../02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 14:20:24 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [48]:
print(exp_L6.feature_metadata)

                                                                                          _feature_id
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o...  k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;...
k__Archaea;p__Euryarchaeota;c__Methanobacteria;...  k__Archaea;p__Euryarchaeota;c__Methanobacteria...
k__Bacteria;__;__;__;__;__                                                 k__Bacteria;__;__;__;__;__
k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin6...  k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin...
k__Bacteria;p__Acidobacteria;c__Solibacteres;o_...  k__Bacteria;p__Acidobacteria;c__Solibacteres;o...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;c__Acidimicrobii...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;

## Running the L6 model

In [49]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_L6.data
y = exp_L6.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_L6.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [50]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11cb0c0d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [51]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': None, 'max_features': 'log2', 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [52]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 822.37


In [53]:
joblib.dump(gs.best_estimator_, 'models/bone_L6_no_metadata.pkl')

['models/bone_L6_no_metadata.pkl']

In [54]:
bone_L6 = joblib.load('models/bone_L6_no_metadata.pkl')
bone_L6

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L6 model

In [56]:
importances = bone_L6.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_L6.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_L6.feature_metadata
np.savetxt("importances/bone_L6_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_L6.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_L6.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(1104,1))

np.savetxt("importances/bone_L6_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodospirillales;f__Acetobacteraceae;g__Roseococcus	0.015528772689147236
2. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Phyllobacteriaceae;g__Defluvibacter	0.013393542140540362
3. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Caulobacterales;f__Caulobacteraceae;g__	0.013042076094020482
4. k__Bacteria;p__Bacteroidetes;c__Sphingobacteriia;o__Sphingobacteriales;f__Sphingobacteriaceae;g__Sphingobacterium	0.011874595943150117
5. k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus	0.010704978441390038
6. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Phyllobacteriaceae;g__	0.010626957220948597
7. k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Nocardioidaceae;g__Aeromicrobium	0.010534233195531099
8. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Sphingom

## Import rarefied table collapsed at L7

In [57]:
exp_L7 = ca.read_amplicon('/Users/heatherdeel/Dropbox/PMI_3_analyses/bone/01_16S/01_qiime2_analysis/feature_tables/collapsed_tables/frag_ins_noChloMito_17098_L7_table.biom', '../02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 15:00:40 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [58]:
print(exp_L7.feature_metadata)

                                                                                          _feature_id
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o...  k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;...
k__Archaea;p__Euryarchaeota;c__Methanobacteria;...  k__Archaea;p__Euryarchaeota;c__Methanobacteria...
k__Bacteria;__;__;__;__;__;__                                           k__Bacteria;__;__;__;__;__;__
k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin6...  k__Bacteria;p__Acidobacteria;c__DA052;o__Ellin...
k__Bacteria;p__Acidobacteria;c__Solibacteres;o_...  k__Bacteria;p__Acidobacteria;c__Solibacteres;o...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Acidobacteria;c__[Chloracidobact...  k__Bacteria;p__Acidobacteria;c__[Chloracidobac...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;c__Acidimicrobii...
k__Bacteria;p__Actinobacteria;c__Acidimicrobiia...  k__Bacteria;p__Actinobacteria;

## Running the L7 model

In [59]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_L7.data
y = exp_L7.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_L7.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [60]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11dca40d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [61]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 16, 'max_features': 'log2', 'min_samples_split': 0.1, 'min_weight_fraction_leaf': 0.0001}


In [62]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 822.28


In [63]:
joblib.dump(gs.best_estimator_, 'models/bone_L7_no_metadata.pkl')

['models/bone_L7_no_metadata.pkl']

In [64]:
bone_L7 = joblib.load('models/bone_L7_no_metadata.pkl')
bone_L7

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=16, max_features='log2', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.1, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the L7 model

In [66]:
importances = bone_L7.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_L7.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_L7.feature_metadata
np.savetxt("importances/bone_L7_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_L7.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_L7.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(1342,1))

np.savetxt("importances/bone_L7_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Phyllobacteriaceae;g__;s__	0.012460536736487698
2. k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Corynebacteriaceae;g__Corynebacterium;s__	0.01165024906309782
3. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Caulobacterales;f__Caulobacteraceae;g__;s__	0.011563001160071463
4. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__Rhizobiaceae;g__Shinella;s__	0.011199123907971294
5. k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Planococcaceae;g__Sporosarcina;s__	0.011058586882234728
6. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhizobiales;f__;g__;s__	0.01098606950938838
7. k__Bacteria;p__Bacteroidetes;c__[Saprospirae];o__[Saprospirales];f__Chitinophagaceae;g__;s__	0.01061631127773758
8. k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Sphingomonadales;f__Sphingomonadaceae;g__Sphingopyxis;s__	0.010522683869193498
9

## Import rarefied table at the ASV level

In [67]:
exp_ASV = ca.read_amplicon('../01_qiime2_analysis/feature_tables/exported-feature-tables/frag_ins_filtered_noChloMito_17098_table.biom', '../02_metadata/maps/map3.txt', min_reads=0, normalize = None)


2020-06-26 15:37:47 WARNING These have metadata but do not have data - dropped (51): {'11553.CSU.extraction.blank1', '11553.BLANKrib.11E', '11553.BLANKrib.10D', '11553.CSU.extraction.blank10', '11553.BLANKrib.10G', '11553.CSU.extraction.blank8', '11553.BLANKrib.10B', '11553.CSU.extraction.blank6', '11553.BLANKrib.9A', '11553.BLANKrib.11G', '11553.CSU.extraction.blank5', '11553.BLANKrib.12F', '11553.BLANKrib.12H', '11553.BLANKrib.9B', '11553.CSU.extraction.blank7', '11553.SHSU.STAFS2016.011.L09', '11553.BLANKrib.11H', '11553.BLANKrib.10F', '11553.CSU.extraction.blank15', '11553.BLANKrib.5B', '11553.CSU.extraction.blank3', '11553.BLANKrib.9D', '11553.BLANKrib.9H', '11553.BLANKrib.9C', '11553.BLANKrib.12A', '11553.STAFS.2016.064.L12', '11553.BLANKrib.11D', '11553.CSU.extraction.blank13', '11553.CSU.extraction.blank14', '11553.CSU.extraction.blank12', '11553.BLANKrib.10A', '11553.CSU.extraction.blank11', '11553.BLANKrib.11C', '11553.BLANKrib.12G', '11553.BLANKrib.12E', '11553.CSU.extractio

In [68]:
exp_ASV.sample_metadata['host_subject_id'].value_counts()

STAFS2016.065    8
STAFS2016.011    8
STAFS2016.067    8
STAFS2016.007    7
STAFS2016.064    7
STAFS2016.024    7
Name: host_subject_id, dtype: int64

In [69]:
exp_ASV.sample_metadata['ADD_0']

#SampleID
11553.SHSU.STAFS2016.024.R09      708.0555556
11553.STAFS.2016.065.L12          2268.055556
11553.STAFS.2016.067.R12          2414.444444
11553.SHSU.STAFS2016.007.L09      1911.111111
11553.SHSU.STAFS2016.011.L08      1301.666667
11553.STAFS.2016.065.R11          4756.388889
11553.STAFS.2016.065.R09          3456.111111
11553.SHSU.STAFS2016.024.L12      4285.833333
11553.SHSU.STAFS2016.007.L11      2584.444444
11553.SHSU.STAFS2016.011.L10      3777.222222
11553.STAFS.2016.064.R10          3456.111111
11553.SHSU.STAFS2016.024.R11      2584.444444
11553.STAFS.2016.067.L11.june     5201.388889
11553.SHSU.STAFS2016.007.R12      1301.666667
11553.STAFS.2016.065.R10          2804.722222
11553.STAFS.2016.064.R09          592.7777778
11553.STAFS.2016.067.L10          3623.055556
11553.SHSU.STAFS2016.011.R12      708.0555556
11553.STAFS.2016.064.R11          1300.555556
11553.SHSU.STAFS2016.011.L11      4285.833333
11553.SHSU.STAFS2016.007.L10      4821.388889
11553.STAFS.2016.065.R08

## Running the ASV model

In [70]:
# groupKfold = 6, will leave out one body for each model
gkf = GroupKFold(6)

X = exp_ASV.data
y = exp_ASV.sample_metadata['ADD_0']
y = (y.astype(float))

groups = exp_ASV.sample_metadata['host_subject_id']

# used to test the param grid for parameter tuning
# use the output of this estimator below for input into new estimator (not commented out)
param_grid = {"max_depth": [4, 8, 16, None],
              "max_features": ['sqrt', 'log2', 0.1],
              "min_samples_split": [0.001, 0.01, 0.1],
              "min_weight_fraction_leaf": [0.0001, 0.001, 0.01],
              "bootstrap": [True, False]}

#param_grid = {"max_depth": [8],
#          "max_features": [0.1],
#          "min_samples_split": [0.001],
#          "min_weight_fraction_leaf": [0.0001],
#          "bootstrap": [False]}

rf = RandomForestRegressor(n_estimators=1000, random_state=999, criterion='mae')
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=1)

In [71]:
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11dc708d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mae', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_es...
                                             oob_score=False, random_state=999,
                                             verbose=0, warm_start=False

In [72]:
# this line is used when run with the first param grid to determine what the best parameters are for tuning
print(gs.best_params_)

{'bootstrap': False, 'max_depth': 8, 'max_features': 0.1, 'min_samples_split': 0.001, 'min_weight_fraction_leaf': 0.0001}


In [73]:
print('The best mean absolute error is', round(-gs.best_score_,2))

The best mean absolute error is 793.33


In [74]:
joblib.dump(gs.best_estimator_, 'models/bone_ASV_no_metadata.pkl')

['models/bone_ASV_no_metadata.pkl']

In [75]:
bone_ASV = joblib.load('models/bone_ASV_no_metadata.pkl')
bone_ASV

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=8, max_features=0.1, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.001, min_weight_fraction_leaf=0.0001,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

## Determine important features of the ASV model

In [77]:
importances = bone_ASV.feature_importances_
std = np.std([tree.feature_importances_ for tree in bone_ASV.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

feature_metadata = exp_ASV.feature_metadata
np.savetxt("importances/bone_ASV_no_metadata_features.csv", feature_metadata, delimiter=",", fmt='%s')


#print the important ids in order
general_importances = []
count = 0
print("Feature:\t\t\t\tImportance:")
for i in indices:
    general_importances += (exp_ASV.feature_metadata.index.values[i], importances[indices[count]])
    if count < 25:
        print(str(count+1)+". "+str(exp_ASV.feature_metadata.index.values[i])+"\t"+str(importances[indices[count]]))
    count += 1
    
general_importances_df = pd.DataFrame(np.array(general_importances).reshape(9442,1))

np.savetxt("importances/bone_ASV_no_features_importances.csv", general_importances_df, delimiter=",", fmt='%s')

Feature:				Importance:
1. TACGAAGGGGGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGATTGGTCAGTTAGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTTTAATACTGCCGGTCTAGAGTTCGAGAGAGGTGAGTGGAATTCCGAG	0.039635896500106335
2. TACGAAGGGGGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGATCGATCAGTTAGGGGTGAAATCCCAGGGCTCAACCCTGGAACTGCCTCTAATACTGTCGATCTCGAGTTCGAGAGAGGTGAGTGGAATTCCGAG	0.03457431648811695
3. TACGTAGGGTGCGAGCGTTGTCCGGAATTACTGGGCGTAAAGAGCTCGTAGGTGGTTTGTCGCGTCGTCTGTGAAATTCCGGGGCTTAACTCCGGGCGTGCAGGCGATACGGGCAAACTGGAGTGCTGTAGGGGAGACTGGAATTCCTGG	0.033536738848873725
4. TACGAAGGGGGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGATATTTAAGTCAGGGGTGAAATCCCGGGGCTCAACCCCGGAACTGCCTTTGATACTGGGTATCTGGAGTATGGAAGAGGTAAGTGGAATTCCGAG	0.029621851722557612
5. TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGATTGGTCAGTTAGGGGTGAAATCCCAGAGCTCAACTCTGGAACTGCCCTTAATACTGCCAGTCTAGAGTCCGGAAGAGGTGAGTGGAACTCCTAG	0.02253507776194126
6. TACGAAGGGGGCTAGCGTTGCTCGGAATCACTGGGCGTAAAGCGCACGTAGGCGGACTCTTAAGTCGGTGGTGAAATCCTGGAGCTCAACTCCAGAACTG